In [1]:
!nvidia-smi

Wed Feb 19 19:53:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

**Model from Hugging Face** 
"https://huggingface.co/Helsinki-NLP/opus-mt-en-fr"

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"

**Dataset from Hugging Face**
https://huggingface.co/datasets/arielogg/anki_globalvoices_en_fr

In [5]:
raw_datasets = load_dataset("arielogg/anki_globalvoices_en_fr")

README.md:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/74.5M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/9.28M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/9.33M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/439064 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/54883 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/54884 [00:00<?, ? examples/s]

In [6]:
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 439064
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 54883
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 54884
    })
})

In [7]:
raw_datasets['train'][2]

{'source': "tom's flight will be arriving early.",
 'target': 'le vol de tom va atterrir plus tôt que prévu.'}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[10537, 2, 67, 32, 15, 5776, 145, 0], [160, 32, 1036, 5776, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [10]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["le vol de tom va atterrir plus tôt que prévu."]))

{'input_ids': [[19, 2730, 5, 49, 10045, 740, 33158, 139, 65, 5741, 29, 2493, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
print(raw_datasets["train"].column_names)

['source', 'target']


In [12]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "fr"

def preprocess_function(examples):
    # Access the source and target fields directly
    inputs = examples["source"]  # Input sentences
    targets = examples["target"]  # Target sentences

    # Tokenize the inputs
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Tokenize the targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    # Add tokenized labels to the inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
preprocess_function(raw_datasets["train"][:5])

{'input_ids': [[1394, 2, 147, 232, 224, 4315, 273, 24, 13485, 64, 98, 30667, 7, 4, 16952, 28, 79, 2737, 30, 27486, 33, 154, 86, 17808, 10, 1787, 15, 6877, 7, 11610, 1162, 57, 6397, 11610, 1162, 3, 0], [12249, 37, 15, 191, 3482, 94, 9543, 18, 4, 3086, 2090, 7639, 1071, 670, 75, 62, 10790, 37, 4, 25084, 7, 49, 1, 226, 20107, 108, 720, 30135, 1, 3, 0], [12, 122, 6, 9, 6135, 86, 45, 23138, 1834, 3, 0], [18, 4, 703, 2, 599, 2308, 26, 192, 1594, 48, 235, 599, 69, 12393, 7, 270, 599, 69, 1905, 3, 0], [22268, 36702, 15, 11230, 7417, 11804, 37, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[1416, 6, 1051, 2, 65, 5, 4847, 1127, 296, 24, 2651, 1234, 13293, 22, 507, 28, 91, 2134,

In [14]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/439064 [00:00<?, ? examples/s]

Map:   0%|          | 0/54883 [00:00<?, ? examples/s]

Map:   0%|          | 0/54884 [00:00<?, ? examples/s]

In [15]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/301M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-fr.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [16]:
batch_size = 32
learning_rate = 3e-5
weight_decay = 0.01
num_train_epochs = 3

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [18]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [19]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [20]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [21]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [22]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)


In [23]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=5)

Epoch 1/5
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
13720/13720 [==============================] - 7522s 543ms/step - loss: 1.1193 - val_loss: 1.0146
Epoch 2/5
13720/13720 [==============================] - 7430s 542ms/step - loss: 1.0019 - val_loss: 1.0027
Epoch 3/5
13720/13720 [==============================] - 7462s 544ms/step - loss: 0.9374 - val_loss: 1.0013
Epoch 4/5
13720/13720 [==============================] - 7447s 543ms/step - loss: 0.8882 - val_loss: 1.0006
Epoch 5/5
13720/13720 [==============================] - 7449s 543ms/step - loss: 0.8463 - val_loss: 1.0078


In [24]:
# Evaluate the model on the validation dataset
evaluation_results = model.evaluate(validation_dataset)

# Print Validation Loss & Accuracy
print(f"Validation Loss: {evaluation_results:.4f}")


1716/1716 [==============================] - 375s 218ms/step - loss: 1.0078
Validation Loss: 1.0078


In [25]:
import sacrebleu
from transformers import pipeline

# Load trained translation model
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

# Sample test sentences
test_sentences = [
    "Hello, how are you?",
    "Artificial intelligence is transforming industries.",
    "Where can I find the nearest coffee shop?",
    "Machine learning is a subset of AI."
]

# Human reference translations (Ground truth)
true_translations = [
    "Bonjour, comment ça va ?",
    "L'intelligence artificielle transforme les industries.",
    "Où puis-je trouver le café le plus proche ?",
    "L'apprentissage automatique est un sous-ensemble de l'IA."
]

# Generate translations from model
predicted_translations = [translator(sentence)[0]['translation_text'] for sentence in test_sentences]

# Compute BLEU Score
bleu_score = sacrebleu.corpus_bleu(predicted_translations, [true_translations])
print(f"BLEU Score: {bleu_score.score:.2f}")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


BLEU Score: 88.14


In [26]:
model.save_pretrained("model")
tokenizer.save_pretrained("model")

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:388: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}
  warnings.warn(


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.json',
 'model/source.spm',
 'model/target.spm',
 'model/added_tokens.json')

In [27]:
!zip -r model.zip model

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: model/ (stored 0%)
  adding: model/special_tokens_map.json (deflated 35%)
  adding: model/generation_config.json (deflated 43%)
  adding: model/config.json (deflated 61%)
  adding: model/source.spm (deflated 49%)
  adding: model/tf_model.h5 (deflated 7%)
  adding: model/tokenizer_config.json (deflated 68%)
  adding: model/target.spm (deflated 50%)
  adding: model/vocab.json (deflated 70%)
